In [1]:
import warnings  # Hide all warnings here
warnings.filterwarnings("ignore")
import mne
mne.utils.set_log_level('error')
import numpy as np
import pandas as pd
from functools import reduce
import local
from models.preprocessing import utils
from joblib import Parallel,delayed

In [2]:
d = "../../data/Julia/New_Datasets/RawData_new/"
d = "../../data/reinhartlab/multimodal/"
ext_in = ".raw.fif.gz"
ext_out = "-epo.fif.gz"
box = utils.select_dataset(d,ext_in,ext_out,show_all=True); box

#### Trial, block & condition declaration

The following dicts are example configurations for declarative trial and block segmentation.

In [3]:
# For probabilistic
cond_train = {
            'condition': 'train',
            'block': { # DONE
                'train': {
                    'begin':'Stim/100',
                    'end':'Stim/101'
            }}, 
            'trial': { # DONE
                'begin':'Stim/5',
                'end':'Stim/6'
            },
            'stimulus': { # DONE
                'begin':'Stim/12',
                'end':'Stim/13',
                'types': {
                    'main': {'AB':'Stim/14','CD':'Stim/15','EF':'Stim/16'},
                    'aux':{'ltr':'Stim/41','rtl':'Stim/42','correct':'Stim/51','incorrect':'Stim/52'}
            }},
            'response': {
                'begin': 'Stim/30',
                'types': {
                    'main':{'correct':'Stim/34','incorrect':'Stim/31'},
                    'aux': {'left':'Stim/37','right':'Stim/38'}
            }},
            'feedback': {
                'begin': 'Stim/20',
                'types': {
                    'main':{'positive': 'Stim/22','negative':'Stim/23','neutral':'Stim/24'}
            }},
            'reaction': {
                'begin':['Stim/12'],
                'end':['Stim/30']
            },
            'require': ['Stim/14','Stim/15','Stim/16']
}
cond_test = {
        'condition': 'test',
            'block': {
                'test': {
                    'begin':'Stim/102',
                    'end':'Stim/103'
            }},
            'trial': {
                'begin':'Stim/17',
                'end':'Stim/18'
            },
            'stimulus': {
                'begin':'Stim/12',
                'end':'Stim/13',
                'types': {
                    'main': {'GG':'Stim/81','NG':'Stim/82','NN':'Stim/83'},
                    'aux': {'A':'Stim/71','B':'Stim/72','C':'Stim/73','D':'Stim/74','E':'Stim/75','F':'Stim/76'}
            }},
            'response': {
                'begin': 'Stim/30',
                'types': {
                    'main': {'correct': 'Stim/91','incorrect':'Stim/90'},
                    'aux': {'left':'Stim/37','right':'Stim/38'}
            }},
            'feedback': {
                'types': {
                    'main': {'positive': 'Stim/91','negative':'Stim/90'}
            }},
            'reaction': {
                'begin':['Stim/12'],
                'end':['Stim/30']
            },
            'require': ['Stim/81','Stim/82','Stim/83']
}
# For multimodal
cond_visual = {
            'condition':'visual',
            'trial': {
                'begin':'Stim/S  9',
                'end':'Stim/S 10'
            },
            'stimulus': {
                'begin':'Stim/S 14',
                'end':'Stim/S 15',
                'types': {
                    'main': {
                        '0':'Stim/S 20',
                        '45':'Stim/S 21',
                        '90':'Stim/S 22',
                        '135':'Stim/S 23'
                }}},
            'block': {
                'visual/right': {
                    'begin':'Stim/S102',
                    'end':'Stim/S104'
                },
                'visual/left': {
                    'begin':'Stim/S108',
                    'end':'Stim/S110'
            }},
            'response': {
                'types': {
                    'main': {
                        'R/left':'Stim/S 60','R/down':'Stim/S 61','R/right':'Stim/S 62','R/up':'Stim/S 63',
                        'L/left':'Stim/S 64','L/down':'Stim/S 65','L/right':'Stim/S 66','L/up':'Stim/S 67'
            }}},
            'feedback': {
                'types': {
                    'main': {
                        'positive': 'Stim/S 50','negative':'Stim/S 52','neutral':'Stim/S 54'
            }}},
            'reaction': {
                'begin':['Stim/S 14'],
                'end':['Stim/S 60','Stim/S 61','Stim/S 62','Stim/S 63','Stim/S 64','Stim/S 65','Stim/S 66','Stim/S 67']
            },
            'require': ['Stim/S 20','Stim/S 21','Stim/S 22','Stim/S 23']
}
cond_auditive = {
            'condition':'auditive',
            'block': {
                'auditive/left': {
                    'begin':'Stim/S105',
                    'end':'Stim/S107'
                },
                'auditive/right': {
                    'begin':'Stim/S111',
                    'end':'Stim/S113'
            }},
            'trial': {
                'begin':'Stim/S  9',
                'end':'Stim/S 10'
            },
            'stimulus': {
                'begin':'Stim/S 14',
                'end':'Stim/S 15',
                'types': {
                    'main':{
                        '250':'Stim/S 16',
                        '500':'Stim/S 17',
                        '1000':'Stim/S 18',
                        '2000':'Stim/S 19'
            }}},
            'response': {
                'types': {
                    'main': {
                        'R/left':'Stim/S 60','R/down':'Stim/S 61','R/right':'Stim/S 62','R/up':'Stim/S 63',
                        'L/left':'Stim/S 64','L/down':'Stim/S 65','L/right':'Stim/S 66','L/up':'Stim/S 67'
            }}},
            'feedback': {
                'types': {
                    'main': {
                        'positive': 'Stim/S 50','negative':'Stim/S 52','neutral':'Stim/S 54'
            }}},
            'reaction': {
                'begin':['Stim/S 14'],
                'end':['Stim/S 60','Stim/S 61','Stim/S 62','Stim/S 63','Stim/S 64','Stim/S 65','Stim/S 66','Stim/S 67']
            },
            'require': ['Stim/S 16','Stim/S 17','Stim/S 18','Stim/S 19']
}

conditions = [cond_visual,cond_auditive]

#### Segment data and save results

Parse metadata and apply trial segmentation. 

In [4]:
invert_dict = lambda dct: {v:k for k,v in dct.items()}
get_all_types = lambda dct: dict(**dct.get('types',{}).get('main',{}),**dct.get('types',{}).get('aux',{}))
get_types_by_trial = lambda events, conf, lock: np.array(['/'.join([invert_dict(get_all_types(conf.get(lock,{}))).get(k) for k in e.keys() if k in get_all_types(conf.get(lock,{})).values()]) for e in events])

def find_blocks(x,y):
    a = x[2]; b = y; c = x[1]; d=x[0]
    if a[0]==b[0]: # block types
        if a[1]=='begin': # block position
            d.update({a[0]:d.get(a[0],0)+1}) # block number
            c = c + [(a[0],d[a[0]],a[2],b[2])] # add block to list
        elif b[1]=='end':
            d.update({b[0]:d.get(b[0],0)+1})
            c = c + [(b[0],d[b[0]],a[2],b[2])]
    else:
        if a[1]=='begin' and b[1]=='begin':
            d.update({a[0]:d.get(a[0],0)+1}) # block number
            c = c + [(a[0],d[a[0]],a[2],b[2])] # add block to list
        elif a[1]=='end' and b[1]=='end':
            d.update({b[0]:d.get(b[0],0)+1})
            c = c + [(b[0],d[b[0]],a[2],b[2])]
    if b[2]==np.infty:
        return c
    else:
        return (d,c,b)
    
def get_trial(events,einvs,begin_code,end_code,sample):
    begins = events[:sample]
    begins = begins[begins[:,2]==begin_code ,:]
    begins = begins[-1][0] if len(begins) else np.infty
    ends = events[sample:]
    ends = ends[ends[:,2]==end_code ,:]
    ends = ends[0][0] if len(ends) else 0
    events = events[np.logical_and(events[:,0]>=begins,events[:,0]<=ends)]
    return {einvs[e[2]]:e[0] for e in events}


def get_metadata(raw,cond,lock='stimulus'):
    """
    Use Condition definition to build metadata dataframe from MNE Raw Object.

    :param raw: RawType object
    :param cond: Condition definition in Dict/Json format (dict)
    :returns: Dataframe for use as Metadata object
    """
    evts,eids = mne.events_from_annotations(raw)
    einvs = {v:k for k,v in eids.items()}
    
    locks = cond.get(lock,{}).get('types',{}).get('main',{})
    lock_codes = [eids.get(v,-1) for v in locks.values()]
    lock_sample = np.array([[e[0],i,e[2]] for i,e in enumerate(evts) if e[2] in lock_codes])
    lock_type = np.array([lock for e in lock_sample[:,2]])

    block_codes = {eids[w]:(k,l) for k,v in cond.get('block',{}).items() for l,w in v.items() if w in eids.keys()}    
    block_events = [[block_codes[e[2]][0],block_codes[e[2]][1],e[0]]for e in evts if e[2] in block_codes.keys()]
    blocks = reduce(find_blocks,block_events+[['','begin',np.infty]],({},[],['','end',0]))
    
    block_idx = [np.logical_and(lock_sample[:,0]<b[3],lock_sample[:,0]>b[2]) for b in blocks]
    block_type = reduce(np.char.add,[np.where(i,b[0],'') for i,b in zip(block_idx,blocks)],'')
    block_idx = np.sum([i*b[1] for i,b in zip(block_idx,blocks)],axis=0)

    events_by_trials = np.array(
        [get_trial(evts[z[0]:z[2]],einvs,eids.get(cond.get('trial',{}).get('begin','Stim/-1'),-1),eids.get(cond.get('trial',{}).get('end','Stim/-1'),-1),z[1]-z[0])
         for z in 
         [(0,lock_sample[0,1],lock_sample[1,1])]+[z for z in zip(lock_sample[:-2,1],lock_sample[1:-1,1],lock_sample[2:,1])]+[(lock_sample[-2,1],lock_sample[-1,1],len(evts))]]) 

    triggers = np.array([np.max([e.get(t,0) for t in cond.get('reaction',{}).get('begin',['Stim/S -1'])]) for e in events_by_trials])
    responses = np.array([np.min([e.get(t,np.infty) for t in cond.get('reaction',{}).get('end',['Stim/S -1'])]) for e in events_by_trials])
    response_time = (responses-triggers)/raw.info['sfreq']
    
    stimulus_type, response_type, feedback_type = (get_types_by_trial(events_by_trials,cond,l) for l in ['stimulus','response','feedback'])

    #response_valence = 1*np.array(['positive' in r for r in response_valence])-1*np.array(['negative' in r for r in response_valence])

    md = {'LockType':lock_type,'LockSample':lock_sample[:,0],'BlockType':block_type,'BlockIndex':block_idx,'StimulusType':stimulus_type,'ResponseType':response_type,'ResponseTimes':response_time,'FeedbackType':feedback_type,'TrialEvents':events_by_trials}
    return pd.DataFrame(md)

In [5]:
def get_good_trials_from_deci(info_fn,no_trials):
    gd_trials = mne.externals.pymatreader.read_mat(info_fn,ignore_fields=['previous'],variable_names=['data'])['data']['condinfo'][2]-1
    return [r in gd_trials for r in range(no_trials)]
def get_epochs_with_metadata(raw,cond,tmin=-2.5,tmax=3.5,baseline=None,lock='stimulus'):
    evts,eids = mne.events_from_annotations(raw)
    locks = cond.get(lock,{}).get('types',{}).get('main',{})
    metadata = get_metadata(raw,cond,lock)
    epochs = mne.Epochs(raw,evts,{'/'.join([cond.get('condition','#'),k]).replace('#/',''):eids.get(v,-1) for k,v in locks.items() if v in eids.keys()},tmin=tmin,tmax=tmax,baseline=baseline,detrend=1,metadata=metadata,preload=True)
    return epochs[epochs.metadata.TrialEvents.map(lambda d: np.any([k in cond.get('require',d.keys()) for k in d.keys()]))]

In [6]:
conditions = [cond_visual,cond_auditive]
# TODO: use joblib parallelization
for f in [d+c+ext_in for c in utils.get_selection(box)][-3:]:
    raw = mne.io.read_raw_fif(f,preload=True)
    evts,eids = mne.events_from_annotations(raw)
    for lock in ['stimulus','response','feedback']:
        epochs = [get_epochs_with_metadata(raw,c,lock=lock) for c in conditions
                  if np.any([x in eids.keys() for x in c.get(lock,{}).get('types',{}).get('main',{}).values()])]
        epochs = mne.concatenate_epochs([e for e in epochs if len(e)],add_offset=False)
        epochs.save(f[:-len(ext_in)]+'.'+lock+ext_out,overwrite=True)
        print(f[:-len(ext_in)]+'.'+lock+ext_out)

../../data/reinhartlab/multimodal/cg\Raw\MultiModal_EEG_091CG.stimulus-epo.fif.gz
../../data/reinhartlab/multimodal/cg\Raw\MultiModal_EEG_091CG.response-epo.fif.gz
../../data/reinhartlab/multimodal/cg\Raw\MultiModal_EEG_091CG.feedback-epo.fif.gz
../../data/reinhartlab/multimodal/cg\Raw\multimodal_eeg_092CG.stimulus-epo.fif.gz
../../data/reinhartlab/multimodal/cg\Raw\multimodal_eeg_092CG.response-epo.fif.gz
../../data/reinhartlab/multimodal/cg\Raw\multimodal_eeg_092CG.feedback-epo.fif.gz
../../data/reinhartlab/multimodal/cg\Raw\multimodal_eeg_105CG.stimulus-epo.fif.gz
../../data/reinhartlab/multimodal/cg\Raw\multimodal_eeg_105CG.response-epo.fif.gz
../../data/reinhartlab/multimodal/cg\Raw\multimodal_eeg_105CG.feedback-epo.fif.gz


In [115]:
tmin=-1;tmax=2;baseline=(-.2,0)
cond = cond_visual; lock = 'stimulus'
evts,eids = mne.events_from_annotations(raw)
locks = cond.get(lock,{}).get('types',{}).get('main',{})
metadata = get_metadata(raw,cond,lock)
ids = {'/'.join([cond.get('condition','#'),k]).replace('#/',''):eids.get(v,-1) for k,v in locks.items() if v in eids.keys()}
epochs = mne.Epochs(raw,evts,ids,tmin=tmin,tmax=tmax,baseline=baseline,detrend=1,preload=True)

In [13]:
box = utils.select_dataset(d,ext_out,show_all=True); box

In [15]:
files = [d+f+ext_out for f in utils.get_selection(box)]

In [ ]:
for f in files:
    epochs = mne.read_epochs(f,preload=False)
    epochs.metadata.to_csv(f[:-len(ext_out)]+'-epo.csv')
    print(f[:-len(ext_out)]+'-epo.csv')

../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_058CG.feedback-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_058CG.response-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_058CG.stimulus-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_069CG.feedback-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_069CG.response-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_069CG.stimulus-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_076CG.feedback-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_076CG.response-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_076CG.stimulus-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_078CG.feedback-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_078CG.response-epo.csv
../../data/reinhartlab/multimodal/cg\Epochs\MulitModal_EEG_078CG.stimulus-epo.csv
../../data/reinh